# Pulling from RIOT API
## Miles Johnson

### Introduction
#### What i wanted to achieve:
#### - Pull my league of legends data from my history of matches.
#### - Convert them into a Dataframe using pandas
#### - Export as a csv file to a database 

In [1]:
#Api key generated from portal
api_key= 'RGAPI-0bbaedde-244c-4cc6-aae8-a374213bf752'

In [2]:
#Url for summonerv4 api
api_url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/archgenesiss'

In [3]:
import requests

In [4]:
# get api url from portal and Have to add api key as arg at end of url
api_url = api_url + "?api_key=" + api_key

In [5]:
api_url

'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/archgenesiss?api_key=RGAPI-0bbaedde-244c-4cc6-aae8-a374213bf752'

In [6]:
#Api request 
resp = requests.get(api_url)

In [7]:
resp.json()

{'id': 'lYSU-8ZAIOwQrEMAJEju7kzJRzuTORlantB8qAIiafolwvc',
 'accountId': 'pZujYry_Ui1qHDpALtlODyVeFe5521itgzeyK2raisrXdL4',
 'puuid': '0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw',
 'name': 'archgenesiss',
 'profileIconId': 5896,
 'revisionDate': 1693896192000,
 'summonerLevel': 106}

In [8]:
#Match api based on puuid entered in portal
api_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw/ids?start=0&count=20'

In [9]:
# Since their are already args in url we have to use & to add an additional one using our api key
api_url = api_url + '&api_key=' + api_key

In [10]:
api_url

'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw/ids?start=0&count=20&api_key=RGAPI-0bbaedde-244c-4cc6-aae8-a374213bf752'

In [11]:
#Api request
resp = requests.get(api_url)

In [12]:
matches = resp.json()

In [13]:
matches

['NA1_4765023061',
 'NA1_4764971657',
 'NA1_4763024815',
 'NA1_4762973640',
 'NA1_4762954541',
 'NA1_4759874005',
 'NA1_4759570819',
 'NA1_4759538503',
 'NA1_4758643215',
 'NA1_4758613462',
 'NA1_4758298016',
 'NA1_4758272488',
 'NA1_4758241138',
 'NA1_4427323326',
 'NA1_4427243194',
 'NA1_4427196900',
 'NA1_4426001154',
 'NA1_4425937034',
 'NA1_4425920109',
 'NA1_4423139932']

In [14]:
match = matches[0]

In [15]:
match

'NA1_4765023061'

In [16]:
#specific match api url using matchid
api_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4765023061'

In [17]:
api_url = api_url + '?api_key=' + api_key
api_url

'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4765023061?api_key=RGAPI-0bbaedde-244c-4cc6-aae8-a374213bf752'

In [18]:
resp = requests.get(api_url)

In [19]:
#Match data
match = resp.json()

In [20]:
match.keys()

dict_keys(['metadata', 'info'])

In [21]:
match['info'].keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [22]:
#My puuid 
puuid = '0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw'

In [23]:
# index of my stats in match data
part_index = match['metadata']['participants'].index(puuid)

In [24]:
match['info']['participants'][part_index]['deaths']

9

In [25]:
#function to get match data using region, match_id, and our apikey should give match data
def get_match_data(region, match_id, api_key):
    api_url = (
    "https://" +
    region + 
    ".api.riotgames.com/lol/match/v5/matches/" +
    match_id + 
    "?api_key=" +
    api_key
    )
    resp = requests.get(api_url)  
    data = resp.json()
    return data

In [26]:
import pandas as pd
import numpy as np

In [27]:
region = 'americas'
api_key = 'RGAPI-0bbaedde-244c-4cc6-aae8-a374213bf752'
puuid = '0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw'

In [28]:
#loop through previous matches, get my stats from each match and store it into a dataframe
match_df = pd.DataFrame()
for match_id in matches:
    match_data = get_match_data(region, match_id, api_key)
    part_index = match_data['metadata']['participants'].index(puuid)
    match_ser = pd.Series(match_data['info']['participants'][part_index]).drop('challenges')
    tmp = pd.DataFrame(match_ser)
    match_df = pd.concat([tmp, match_df], axis=1)
    
   

In [29]:
match_df = match_df.T

In [30]:
match_df = match_df.fillna(0)
match_df = match_df.reset_index().drop('index', axis=1)

In [31]:
#Dataframe of my last 20 games and its stats
match_df

,assists,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,...,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,pushPings,subteamPlacement,totalAllyJungleMinionsKilled,totalEnemyJungleMinionsKilled,visionClearedPings
0,11,0,1,1,12318,14,200,Belveth,0,1,...,0,0,0,0,0,0,0,0,0,0
1,10,0,11,0,18110,17,77,Udyr,0,7,...,0,0,0,0,0,0,0,0,0,0
2,19,0,9,0,13728,15,77,Udyr,0,2,...,0,0,0,0,0,0,0,0,0,0
3,4,0,8,2,10937,13,11,MasterYi,0,1,...,0,0,0,0,0,0,0,0,0,0
4,8,0,14,2,10231,13,77,Udyr,0,3,...,0,0,0,0,0,0,0,0,0,0
5,17,2,17,0,31291,18,131,Diana,0,9,...,0,0,0,0,0,0,0,0,0,0
6,13,0,11,0,14768,16,77,Udyr,0,2,...,0,0,0,0,0,0,0,0,0,0
7,13,0,0,2,17542,17,22,Ashe,0,2,...,0,0,0,0,0,0,0,10,0,0
8,3,0,0,0,10872,13,21,MissFortune,0,1,...,0,0,0,0,0,0,0,4,0,0
9,4,0,0,0,11455,14,21,MissFortune,0,3,...,0,0,0,0,0,0,0,0,0,0


### Finally put my stats into a dataframe and now its time to start storing it into csv files

In [32]:
# list of columns i care about
stats = ['assists', 'kills', 'deaths', 'championName', 'goldEarned', 'goldSpent', 'damageDealtToObjectives', 'itemsPurchased',
         'longestTimeSpentLiving', 'totalMinionsKilled', 'wardsPlaced', 'visionScore', 'teamPosition', 'totalDamageDealt', 'totalDamageDealtToChampions', 
         'totalDamageTaken', 'totalTimeSpentDead', 'win']
    

In [33]:
#updating the frame to only those columns
match_df = match_df[stats]

In [34]:
#Where i want my games to start
match_df = match_df.iloc[7:].reset_index()

In [35]:
match_df = match_df.drop('index', axis= 1)

In [36]:
match_df

,assists,kills,deaths,championName,goldEarned,goldSpent,damageDealtToObjectives,itemsPurchased,longestTimeSpentLiving,totalMinionsKilled,wardsPlaced,visionScore,teamPosition,totalDamageDealt,totalDamageDealtToChampions,totalDamageTaken,totalTimeSpentDead,win
0,13,2,2,Ashe,10870,10208,4254,22,383,141,4,10,BOTTOM,137134,10111,12962,26,False
1,3,5,7,MissFortune,10298,9950,3994,17,533,139,2,4,BOTTOM,76865,13693,17438,185,False
2,4,18,7,MissFortune,15591,12750,6340,17,627,131,8,23,BOTTOM,108280,32188,25559,185,False
3,9,10,10,MissFortune,13741,12400,2353,27,485,190,11,26,BOTTOM,168509,22714,26062,371,False
4,5,6,4,MissFortune,10794,9675,5245,20,836,129,8,15,BOTTOM,90422,19226,10890,100,False
5,5,4,6,Caitlyn,8718,8075,837,12,852,144,6,11,BOTTOM,95927,12396,17535,176,False
6,15,12,5,Tristana,14730,14625,13560,21,763,144,11,30,BOTTOM,127642,25186,24512,193,True
7,8,5,1,Caitlyn,9923,9600,13674,14,965,155,6,9,BOTTOM,103820,14711,6569,30,True
8,0,2,0,Caitlyn,3692,1875,0,4,0,78,1,1,BOTTOM,28461,3511,2324,0,True
9,10,2,6,Caitlyn,11094,10425,11839,18,606,187,9,21,BOTTOM,137092,10799,19064,181,False


In [37]:
#match_df.to_csv('matchdata_clean.csv')